In [105]:
#Import the necessary methods from tweepy library
import tweepy
import math
import csv
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


#These are display options in order to help the dataframe outputs not be truncated
pd.options.display.max_columns = 50 
pd.options.display.max_rows = 50 
pd.options.display.width = 120 
pd.options.display.max_colwidth = 110

from tweepy import OAuthHandler

#Variables that contains the user credentials to access Twitter API 
CONSUMER_KEY = ''
CONSUMER_SECRET =''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''
    
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth)

In [3]:
#This is just to test if the connection was established. Should only have 15 items in this. 
results = api.search(q="%23blues")

In [4]:
#Simple function to print individual tweets
def print_tweet(tweet): 
    print "@%s - %s (%s)" % (tweet.user.screen_name, tweet.user.name, tweet.user.created_at)
    print tweet.text
    
tweet = results[0]
print (print_tweet(tweet))

@matarosario831 - Rosario Mata (2015-10-07 18:36:11)
RT @GDoblesMusic: ¿Un poco de #blues para terminar la semana? Inspirado en el piano les regalo esta linda melodía #BuenViernes https://t.co…
None


In [5]:
results = []

#This is what gets all the tweets. I believe that 5000 is currently the max items you can get from using the REST API for twitter
for tweet in tweepy.Cursor(api.search, q = '%23blues').items(5000):
    results.append(tweet)
    
print len(results)

3870


In [6]:
#This function puts your results into a dataframe that makes it much easier to read and work with. 
#Each column in the dataframe corresponds to the [''] part of it
def process_results(results):
    
    id_list = [tweet.id for tweet in results]
    
    df = pd.DataFrame(id_list, columns=["id"])
    
    #Processing Tweet data 
    df['text'] = [tweet.text for tweet in results]
    df['created_at'] = [tweet.created_at for tweet in results]
    df['retweet_count'] = [tweet.retweet_count for tweet in results]
    df['favorite_count'] = [tweet.favorite_count for tweet in results]
    
    #Processing User Data
    df['user_screen_name'] = [tweet.author.screen_name for tweet in results]
    df['user_name'] = [tweet.author.name for tweet in results]
    df['user_location'] = [tweet.author.location for tweet in results]
    
    return df

df = process_results(results)
print len(df)

3870


In [7]:
#This returns the first twenty items in your dataframe
df.head(20)

,id,text,created_at,retweet_count,favorite_count,user_screen_name,user_name,user_location
0,774432361832325120,RT @GDoblesMusic: ¿Un poco de #blues para term...,2016-09-10 02:20:37,2,0,matarosario831,Rosario Mata,
1,774432213802835968,#IWokeUp @McNeillShona https://t.co/Ofh71bdZOE...,2016-09-10 02:20:02,0,0,Lov3rzLov3You,Michelle #LOV3RZ,Worldwide
2,774431974354288640,#galveston #texas #blues #fuckinword @ Old Qua...,2016-09-10 02:19:05,0,0,manny9000,manny salazar,Houston Texas
3,774431387017474048,#BBKing #BluesLegend Live at the Regal by B.B....,2016-09-10 02:16:45,0,0,HotTrends8,Google Hot Trends,
4,774430888197259264,Listening to Looking Good by Magic Sam on http...,2016-09-10 02:14:46,0,0,Latitude23R,Latitude23 Radio,"Phoenix, AZ"
5,774430346498768896,RT @HunterAHomistek: Stevie Ray Vaughan's best...,2016-09-10 02:12:37,1,0,homistek,Mr marty,Planet earth
6,774430248574324736,#doublebass #bass #bassguitar #jamsession #beb...,2016-09-10 02:12:14,0,1,shulman_susan,susan shulman,
7,774429935138209792,@Marlin Saner Rain For Africa https://t.co/3fk...,2016-09-10 02:10:59,0,0,lorraine_jazz,Lorraine Jazz,"Nancy, Lorraine"
8,774429930969042944,"Now Playing, Little Freddie King — Louisiana T...",2016-09-10 02:10:58,0,0,wnmcradio,WNMC Radio Playlist,"Traverse City, MI"
9,774429767223369728,Listen to my #awardwinningsong #moonblues on #...,2016-09-10 02:10:19,0,0,delaine2511,Vanessa Delaine,Australia


In [9]:
#This gives you the top 15 places in user_location 
top_places = df["user_location"].value_counts()[:15]
print top_places

                                1081
Hatfield, Hertfordshire, UK      223
London, England                  157
Nancy, Lorraine                  130
Phoenix, AZ                      123
Lorraine                          92
Worldwide                         51
London                            37
Casablanca, Grand Casablanca      25
Traverse City, MI                 25
Fort Lauderdale                   24
Local, Everywhere!™               23
Los Angeles, CA                   20
Birmingham                        20
Mother Earth                      19
Name: user_location, dtype: int64


In [10]:
#I used this function to help clean up and organize the unstructured data by removing common places to regular state abbreviations
def cleanup(df):  
    new_df = df.replace(to_replace = "Hatfield, Hertfordshire, UK", value = "UK")
    new_df = new_df.replace(to_replace = "United States", value = "US")
    new_df = new_df.replace(to_replace = "London", value = "UK")
    new_df = new_df.replace(to_replace = "London, GB", value = "UK")
    new_df = new_df.replace(to_replace = "London, England", value = "UK")
    new_df = new_df.replace(to_replace = "Traverse City, MI", value = "MI")
    new_df = new_df.replace(to_replace = "Los Angeles, CA", value = "CA")
    new_df = new_df.replace(to_replace = "New York NY (Forest Hills) USA", value = "NY")
    new_df = new_df.replace(to_replace = "Canada", value = "CA")
    new_df = new_df.replace(to_replace = "Los Angeles, California", value = "CA")
    new_df = new_df.replace(to_replace = "Birmingham", value = "UK")
    new_df = new_df.replace(to_replace = "Lorraine", value = "France")
    new_df = new_df.replace(to_replace = "Texas", value = "TX")
    new_df = new_df.replace(to_replace = "Valencia", value = "CA")
    new_df = new_df.replace(to_replace = "East Coast, N.J.", value = "NJ")
    new_df = new_df.replace(to_replace = "Vancouver, British Columbia", value = "Canada")
    new_df = new_df.replace(to_replace = "Longuyon, France", value = "France")
    new_df = new_df.replace(to_replace = "Venezuela", value = "VE")
    new_df = new_df.replace(to_replace = "Bluesville, Illinois", value = "IL")
    new_df = new_df.replace(to_replace = "London, UK", value = "UK")
    new_df = new_df.replace(to_replace = "New York", value = "NY")
    new_df = new_df.replace(to_replace = "Chicago", value = "IL")
    new_df = new_df.replace(to_replace = "London, Ontario", value = "Canada")
    new_df = new_df.replace(to_replace = "Austin, TX", value = "TX")
    new_df = new_df.replace(to_replace = "Sausalito, CA", value = "CA")
    new_df = new_df.replace(to_replace = "Nashville, TN", value = "TN")
    new_df = new_df.replace(to_replace = "España", value = "Spain")
    new_df = new_df.replace(to_replace = "Boston, MA", value = "MA")
    new_df = new_df.replace(to_replace = "Sweden", value = "Sweden")
    new_df = new_df.replace(to_replace = "Bogota, Colombia", value = "Colombia")
    new_df = new_df.replace(to_replace = "Florida", value = "FL")
    new_df = new_df.replace(to_replace = "California", value = "CA")
    new_df = new_df.replace(to_replace = "Illinois", value = "IL")
    new_df = new_df.replace(to_replace = "Bergen County New Jersey", value = "NJ")
    new_df = new_df.replace(to_replace = "Kula, Hawaii", value = "HI")
    new_df = new_df.replace(to_replace = "Chicago, IL", value = "IL")
    new_df = new_df.replace(to_replace = "Ottawa, Ontario", value = "Canada")
    new_df = new_df.replace(to_replace = "Florence, SC", value = "SC")
    new_df = new_df.replace(to_replace = "Hull Uk  (from MemphisTN-USA)", value = "TN")
    new_df = new_df.replace(to_replace = "Seattle, WA", value = "WA")
    new_df = new_df.replace(to_replace = "Michigan, USA", value = "MI")
    new_df = new_df.replace(to_replace = "Fort Lauderdale", value = "FL")
    new_df = new_df.replace(to_replace = "Los Angeles", value = "CA")
    new_df = new_df.replace(to_replace = "Lebanon Oregon USA", value = "OR")
    new_df = new_df.replace(to_replace = "Flowery Branch, GA", value = "GA")
    new_df = new_df.replace(to_replace = "New York, NY", value = "NY")
    new_df = new_df.replace(to_replace = "St. Louis, MO", value = "MO")
    new_df = new_df.replace(to_replace = "Utah", value = "UT")
    new_df = new_df.replace(to_replace = "", value = "US")
    new_df = new_df.replace(to_replace = "Paris, Ile-de-France", value = "France")
    new_df = new_df.replace(to_replace = "United Kingdom", value = "UK")
    new_df = new_df.replace(to_replace = "Toronto, Ontario", value = "Canada")
    new_df = new_df.replace(to_replace = "Georgia", value = "GA")
    new_df = new_df.replace(to_replace = "California, USA", value = "CA")
    new_df = new_df.replace(to_replace = "Iowa, USA", value = "IA")
    new_df = new_df.replace(to_replace = "Albany, OR", value = "OR")
    new_df = new_df.replace(to_replace = "Miami Springs", value = "FL")
    new_df = new_df.replace(to_replace = "Ohio", value = "OH")
    new_df = new_df.replace(to_replace = "Arkansas", value = "AR")
    new_df = new_df.replace(to_replace = "Atlanta, GA", value = "GA")
    new_df = new_df.replace(to_replace = "Massachusetts", value = "MA")
    new_df = new_df.replace(to_replace = "Indiana, USA", value = "IN")
    new_df = new_df.replace(to_replace = "Miami Springs, Florida", value = "FL")
    new_df = new_df.replace(to_replace = "NY, NY", value = "NY")
    new_df = new_df.replace(to_replace = "Long Beach, Cal", value = "CA")
    new_df = new_df.replace(to_replace = "Mississippi", value = "MS")
    new_df = new_df.replace(to_replace = "Wisconsin", value = "WI")
    new_df = new_df.replace(to_replace = "Malibu, CA", value = "CA")
    new_df = new_df.replace(to_replace = "Los Angeles / Hollywood", value = "CA")
    new_df = new_df.replace(to_replace = "Shelby, North Carolina", value = "NC")
    new_df = new_df.replace(to_replace = "Beverly Hills, CA", value = "CA")
    new_df = new_df.replace(to_replace = "Puyallup, WA", value = "WA")
    new_df = new_df.replace(to_replace = "Tawas City, Michigan", value = "MI")
    new_df = new_df.replace(to_replace = "Kentucky", value = "KT")
    new_df = new_df.replace(to_replace = "Pennsylvania", value = "PN")
    new_df = new_df.replace(to_replace = "Hollywood - Los Angeles, CA", value = "CA")
    new_df = new_df.replace(to_replace = "St Louis, Missouri, USA", value = "MS")
    new_df = new_df.replace(to_replace = "New Jersey", value = "NJ")
    new_df = new_df.replace(to_replace = "Connecticut, USA", value = "CT")
    new_df = new_df.replace(to_replace = "Sacramento, CA", value = "CA")
    new_df = new_df.replace(to_replace = "San Rafael, CA", value = "CA")
    new_df = new_df.replace(to_replace = "Houston, TX", value = "TX")
    new_df = new_df.replace(to_replace = "texas", value = "TX")
    new_df = new_df.replace(to_replace = "Phoenix, AZ", value = "AZ")
    return new_df 
    



#if you can't get the function to work just change it back to not being a function
#clean_up(df)
#print df['user_location'].value_counts()[:15]

In [11]:
#Prints out the top 15 places in user_location after clean up has been used on the dataframe
clean = cleanup(df)
print clean['user_location'].value_counts()[:15]

C:\Users\Adrian\Anaconda2\lib\site-packages\pandas\core\common.py:488: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  mask = arr == x


US                              1090
UK                               487
Nancy, Lorraine                  130
AZ                               124
France                           107
CA                                53
Worldwide                         51
IL                                44
MI                                29
FL                                28
NJ                                27
Casablanca, Grand Casablanca      25
Local, Everywhere!™               23
Mother Earth                      19
Sweden                            17
Name: user_location, dtype: int64


In [13]:
#This line was used just in case there were any values in user_location that weren't read in as strings
#new_df.user_location = new_df.user_location.to_string()

In [14]:
#This line below takes the clean dataframe and puts it into a .csv file with a utf-8 encoding
#clean.to_csv('summer8.csv', encoding = 'utf-8')

In [15]:
#This part reads in .csv files from different days
summer = pd.read_csv(os.path.join('summer7.csv'))
summer2 = pd.read_csv(os.path.join('summer8.csv'))
summer3 = pd.read_csv(os.path.join('summer.csv'))

#These lines join the three separate dataframes above and joins them all together into one 
result = summer.append(summer2)
result = result.append(summer3)

In [16]:
#This cleans up the one big dataframe that we now have and outputs the top 15 places 
df = cleanup(result)
print df['user_location'].value_counts()[:15]

US                              1219
UK                               521
AZ                               149
France                           116
CA                               102
MI                                58
IL                                56
Worldwide                         52
TN                                45
Herne Bay Kent CT6                32
Casablanca, Grand Casablanca      28
NY                                21
espelkamp germany                 19
TX                                19
Sweden                            15
Name: user_location, dtype: int64


In [125]:
#This takes all the values in the dataframe that has its user_location in the same state and separates them into their own 
#dataframes
CA = df[df['user_location'] == 'CA']
TX = df[df['user_location'] == 'TX']
AZ = df[df['user_location'] == 'AZ']
MI = df[df['user_location'] == 'MI']
TN = df[df['user_location'] == 'TN']
IL = df[df['user_location'] == 'IL']
USA = df[df['user_location'] == 'US']

In [126]:
#This takes the combined dataframe we had from earlier and puts it into its own .csv file 
df.to_csv('combined.csv', encoding = 'utf-8')

In [137]:
#This function prints the text of the tweet with the max number of retweets as well as max retweet number 
def max_print_state(df): 
    state = str(df['user_location'].iloc[0])
    print state + ": " + df['text'].max()
    print "Retweet count: " + str(df['retweet_count'].max())
    print

max_print_state(CA)
max_print_state(TX)
max_print_state(AZ)
max_print_state(IL)
max_print_state(MI)
max_print_state(TN)

CA: https://t.co/vYclMJhDZu

Some lunchtime #Blues for you
@TheRealBuddyGuy and SRV
Passing fwd = enjoyment💙❗️ https://t.co/nC6elJRpl6
Retweet count: 29.0

TX: So pumped to be a part of @kpboomboom #album #release #music #indie #blues #artist #concert… https://t.co/BrpiKeQqh5
Retweet count: 296.0

AZ: RT @Fender: RT @rtruesdale50: These Fender Blues Deville harps are very cool. Excellent tone and vibe. @Fender #harmonica #blues https://t.…
Retweet count: 41.0

IL: the #blues defined, by @bluesAllstars How #chicago does #music https://t.co/p1Q4S2tfqx
Retweet count: 15.0

MI: Slow Train - Joe Bonamassa | Blues |425300105 #Blues https://t.co/B9wqYx3OJR #Blues
Retweet count: 3.0

TN: https://t.co/L0CegLDUps  #music …https://t.co/g192SopB6J   https://t.co/8mgUvTYl32 …  https://t.co/qpCI2ngfuY  #pop #blues
Retweet count: 11.0



In [148]:
#I used this function to print out the text of a tweet with the retweet_count amount above the percentile defined
def top_tweets(df):
    state = str(df['user_location'].iloc[0])
    print state + ": " + df['text'].max() + '\n' 
    print df.sort('retweet_count', ascending = True)[:5].text
    print

In [149]:
top_tweets(CA)
top_tweets(TX)
top_tweets(AZ)
top_tweets(IL)
top_tweets(TN)
top_tweets(MI)

CA: https://t.co/vYclMJhDZu

Some lunchtime #Blues for you
@TheRealBuddyGuy and SRV
Passing fwd = enjoyment💙❗️ https://t.co/nC6elJRpl6

1        Listening to #JimmyRushing singing the #blues! #NowPlaying "T'aint Nobody's Bizness" https://t.co/uNM7XjL9ab
2428    #morning #blues #bus #ride #september62014 #classic #blackandwhite #glasses #immortal @ Los… https://t.co/...
2315    Especial Entrevista a Jose Luis Acosta Can Blues Pink Tone Jueves 9 PM #BLUES # BluesRock en Las Fronteras...
2089    @AmeriBluesScene @Bluescentric  Nice to see a #Canadian #blues #musician in that list @sweetharrisonk ,unl...
2060    "Slippin and Slidin" at The Malibu Chili Cook-Off Sat #Rock #Blues #guitarsolo #slideguitar… https://t.co/...
Name: text, dtype: object

TX: So pumped to be a part of @kpboomboom #album #release #music #indie #blues #artist #concert… https://t.co/BrpiKeQqh5

1276          Currently on air: TOTAL GRAVEYARD BLUES on https://t.co/11jvnMonEq  #blues #delta #harambe #BLM #Austin
252    

C:\Users\Adrian\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
